In [ ]:
import pandas as pd 
import datetime as dt 
from pandas_datareader import data as pdr 
import matplotlib.pyplot as plt 
import plotly
import plotly.graph_objects as go
import seaborn as sns
import yfinance as yf

## Specify Date range

In [ ]:
end = dt.datetime.now()
start = dt.datetime(2019,1,1)
start, end

stocklist = ['AAPL', 'MSFT', 'AMZN', 'GOOGL', 'META', 'TSLA', 'NVDA', 'JPM', 'JNJ', 'V', 'PG', 'HD', 'UNH', 'MA']

#Define funciton to fetch stock data
def fetch_data(tickerss, start, end, interval ='1d'):
    for ticker in tickerss:
        try:
            df = yf.download(ticker, start, end, interval = interval)
            

            if df.empty:
                print(f"No data returned for {ticker}")
                continue  # Skip to the next ticker if no data
            
            df.index = pd.to_datetime(df.index).strftime('%d-%m-%Y')
            
        except Exception as e:
            print(f"Could not retrieve data for {ticker}: {e}")
    
    return df  # Return the DataFrame with all tickers' data



In [ ]:
#Define function to get user input
def get_user_input(tickerss):
    user_input = input("Enter one or more ticker symbols, type 'all': ").strip()
    if user_input.lower() == 'all':
        return stocklist
    else: 
        user_tickers = [ticker.strip().upper() for ticker in user_input.split(',')]
        valid_tickers = [ticker for ticker in user_tickers if ticker in stocklist]

        if not valid_tickers:
            print("no valid tickers entered. Please try again.")
            return []
        return valid_tickers

In [ ]:
df.index = pd.to_datetime(df.index).strftime('%d-%m-%Y')
df['Adj Close']

In [ ]:
stock_data = fetch_data(stocklist, start, end)

In [ ]:
stock_data.head()

In [ ]:
df

In [ ]:
df['Adj Close'].plot(figsize = (10,8))

In [ ]:
#Get user ticker (specifying parameter overrides need for input feature)
tickerss = get_user_input(stocklist)

In [ ]:
#Fetch stock data for selected tickers


In [ ]:
# stocklist = ['NIO', 'PYPL', 'META','TSLA', 'UBER', 'MANU', 
# 'NFLX', 'GC=F', 'SI=F', 'CL=F', 'ETH-USD', 'SHEL.L', 'BP', 'FRO', 'GLNG', 'TK', 
# 'GRAB', 'GOTO.JK', 'ZOMATO.NS', 'DHER.DE', '3690.HK', 'JMIA']


In [ ]:
#Calculate moving averages for trend detection
#Define function to calculate moving averages
def calculate_moving_averages(stock_data, short_window = 50, long_window = 200):

    short_ma = stock_data.rolling(window = short_window).mean()
    long_ma = stock_data.rolling(window = long_window).mean()
    return short_ma, long_ma

#create dataframe to house moving averages
moving_averages = pd.DataFrame()
bollinger = pd.DataFrame()


In [ ]:
#Calculate Bollinger Bands for volatility and prive level evaluation

def calculate_bollinger_bands(stock_data, window = 20, num_sd = 2):

    rolling_mean = stock_data.rolling(window = window).mean()
    rolling_std = stock_data.rolling(window = window).std()
    upper_band = rolling_mean + (rolling_std * num_sd)
    lower_band = rolling_mean - (rolling_std * num_sd)
    return upper_band, lower_band


#Loop through each ticker and add short/long term ma to the DF
for ticker in stock_data.columns:
    if f"{ticker}_Adj Close" in stock_data.columns:  # Ensure adjusted close exists
        short_ma, long_ma = calculate_moving_averages(stock_data[f'{ticker}_Adj Close'])
        moving_averages[f'{ticker}_Short_MA'] = short_ma
        moving_averages[f'{ticker}_Long_MA'] = long_ma
        
        upper_band, lower_band = calculate_bollinger_bands(stock_data[ticker])
        bollinger[f'{ticker}_Up'] = upper_band
        bollinger[f'{ticker}_Low'] = lower_band

In [ ]:
import plotly.graph_objects as go

def plot_trend_with_indicators(stock_data, moving_averages, bollinger, ticker):
    fig = go.Figure()
    
    # Stock price (using the specified ticker)
    fig.add_trace(go.Scatter(x=stock_data.index, y=stock_data[f'{ticker}_Adj Close'], mode='lines', name=f'{ticker} Price'))
    
    # Short-term moving average
    fig.add_trace(go.Scatter(x=moving_averages.index, y=moving_averages[f'{ticker}_Short_MA'], mode='lines', name='Short-term MA (50 days)', line=dict(dash='dash')))
    
    # Long-term moving average
    fig.add_trace(go.Scatter(x=moving_averages.index, y=moving_averages[f'{ticker}_Long_MA'], mode='lines', name='Long-term MA (200 days)', line=dict(dash='dot')))
    
    # Bollinger Bands
    fig.add_trace(go.Scatter(x=bollinger.index, y=bollinger[f'{ticker}_Upper_Band'], mode='lines', name='Upper Band', line=dict(color='red', dash='dash')))
    fig.add_trace(go.Scatter(x=bollinger.index, y=bollinger[f'{ticker}_Lower_Band'], mode='lines', name='Lower Band', line=dict(color='green', dash='dash')))
    

    
    # Volume (if it exists in stock_data)
    volume_column = f'{ticker}_Volume'
    if volume_column in stock_data.columns:
        fig.add_trace(go.Bar(x=stock_data.index, y=stock_data[volume_column], name=f'{ticker} Volume', yaxis='y2', opacity=0.4))

    # Update layout to show two y-axes (one for price, one for volume)
    fig.update_layout(
        title=f'{ticker} Trend Analysis with Indicators',
        xaxis_title='Date',
        yaxis_title='Price',
        yaxis2=dict(title='Volume', overlaying='y', side='right', showgrid=False),
        legend=dict(x=0, y=1.1)
    )
    
    fig.show()



for ticker in selected:
    if f"{ticker}_Adj Close" in stock_data.columns:
        plot_trend_with_indicators(stock_data, moving_averages, bollinger, ticker)
    else: 
        print(f"No data available for {ticker}.")

In [ ]:
# #Calculate RSI to assess overbought or oversold conditions
# def calculate_rsi(stock_data, window=14):
#     """
#     Calculate the Relative Strength Index (RSI).
#     """
#     delta = stock_data.diff()
#     gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
#     loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
#     rs = gain / loss
#     rsi = 100 - (100 / (1 + rs))
#     return rsi

# # Calculate RSI for each stock
# rsi_values = {ticker: calculate_rsi(data[ticker]) for ticker in top_15_tickers}


In [ ]:
#Identify price dips below moving averages and bollinger
#Define functions to find dips
def find_dips(stock_data, short_ma, threshold=-0.05):
    deviation = (stock_data - short_ma) / short_ma
    dips = stock_data[deviation < threshold]
    return dips

In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = stock_data.index, y = stock_data['META_Adj Close'], mode = 'lines' ))
fig.update_layout(title=f'{ticker} Trend Analysis with Indicators', xaxis_title='Date', yaxis_title='Price')

fig.show()

In [ ]:
fig1 = go.Figure()

fig1.add_trace(go.Scatter(x=short_ma.index, y=short_ma, mode='lines', name='Short-term MA (50 days)', line=dict(dash='dash')))

fig1.show()

In [ ]:
# Bollinger Bands
fig2 = go.Figure()

fig2.add_trace(go.Scatter(x=upper_band.index, y=upper_band, mode='lines', name='Upper Band', line=dict(color='red', dash='dash')))
fig2.add_trace(go.Scatter(x=lower_band.index, y=lower_band, mode='lines', name='Lower Band', line=dict(color='green', dash='dash')))

fig2.show()

In [ ]:
# # Original list of units
# unit_list = ['GC=F', 'SI=F', 'CL=F']

# # Dictionary mapping old unit names to new names
# unit_mapping = {
#     'GC=F': 'GOLD',
#     'SI=F': 'SILVER',
#     'CL=F': 'CRUDE'
# }

# # Using list comprehension to rename units
# stocklist = [unit_mapping.get(stock, stock) for stock in stocklist]

# print(stocklist)


In [ ]:
# ClVol = stock_data[[col for col in stock_data.columns if 'Adj Close' in col or 'Volume' in col]]
# ClVol.head()

In [ ]:
# ClVol = stock_data[[col for col in stock_data.columns if 'Adj Close' in col or 'Volume' in col]]
# ClVol.tail(15)